# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f306099c1f0>
├── 'a' --> tensor([[ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120]])
└── 'x' --> <FastTreeValue 0x7f306099c5e0>
    └── 'c' --> tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                        [-1.9209,  0.1798,  0.7683, -1.1547],
                        [ 0.7718,  0.4371,  0.8036, -0.2444]])

In [4]:
t.a

tensor([[ 1.4126,  0.7742,  0.0023],
        [ 1.5242, -0.4225,  0.5120]])

In [5]:
%timeit t.a

80 ns ± 0.111 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f306099c1f0>
├── 'a' --> tensor([[0.3196, 1.5749, 1.1784],
│                   [0.0320, 1.1575, 0.0840]])
└── 'x' --> <FastTreeValue 0x7f306099c5e0>
    └── 'c' --> tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                        [-1.9209,  0.1798,  0.7683, -1.1547],
                        [ 0.7718,  0.4371,  0.8036, -0.2444]])

In [7]:
%timeit t.a = new_value

85.7 ns ± 0.241 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120]]),
    x: Batch(
           c: tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                      [-1.9209,  0.1798,  0.7683, -1.1547],
                      [ 0.7718,  0.4371,  0.8036, -0.2444]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4126,  0.7742,  0.0023],
        [ 1.5242, -0.4225,  0.5120]])

In [11]:
%timeit b.a

71.8 ns ± 0.998 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8520,  0.6893,  0.3145],
               [-0.3388,  0.8294,  1.5606]]),
    x: Batch(
           c: tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                      [-1.9209,  0.1798,  0.7683, -1.1547],
                      [ 0.7718,  0.4371,  0.8036, -0.2444]]),
       ),
)

In [13]:
%timeit b.a = new_value

598 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

969 ns ± 4.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

165 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2fb66c8cd0>
├── 'a' --> tensor([[[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]],
│           
│                   [[ 1.4126,  0.7742,  0.0023],
│                    [ 1.5242, -0.4225,  0.5120]]])
└── 'x' --> <FastTreeValue 0x7f2fb66c8c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

36.8 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2fbd28b2b0>
├── 'a' --> tensor([[ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120],
│                   [ 1.4126,  0.7742,  0.0023],
│                   [ 1.5242, -0.4225,  0.5120]])
└── 'x' --> <FastTreeValue 0x7f2fb66c8ca0>
    └── 'c' --> tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                        [-1.9209,  0.1798,  0.7683, -1.1547],
                 

In [23]:
%timeit t_cat(trees)

36.6 µs ± 615 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]],
       
               [[ 1.4126,  0.7742,  0.0023],
                [ 1.5242, -0.4225,  0.5120]]]),
    x: Batch(
           c: tensor([[[-0.5477, -1.8098,  0.0351, -0.4242],
                       [-1.9209,  0.1798,  0.7683, -1.1547],
                       [ 0.7718,  0.4371,  0.8036, -0.2444]],
         

In [26]:
%timeit Batch.stack(batches)

91.6 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120],
               [ 1.4126,  0.7742,  0.0023],
               [ 1.5242, -0.4225,  0.5120]]),
    x: Batch(
           c: tensor([[-0.5477, -1.8098,  0.0351, -0.4242],
                      [-1.9209,  0.1798,  0.7683, -1.1547],
                      [ 0.7718,  0.4371,  0.8036, -0.2444],
                      [-0.5477, -1.8098,  0.0351, -0.4242],
                      [-1.9209,  

In [28]:
%timeit Batch.cat(batches)

166 µs ± 672 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

383 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
